In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from utils.fmodule import FModule
import torch.nn.functional as F

In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class Model(FModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(3136, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def encoder(self, x):
        x = x.view((x.shape[0],28,28))
        x = x.unsqueeze(1)
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        return x

    def decoder(self, x):
        x = self.fc2(x)
        return x
    
    def pred_and_rep(self, x):
        e = self.encoder(x)
        o = self.decoder(e)
        return o, e

model = Model().to(device)

Using cuda device


In [3]:
from algorithm.utils.alg_utils.alg_utils import get_ultimate_layer, KDR_loss, get_module_from_model

In [4]:
import copy

classifiers = []
for i in range(5):
    classifiers.append(copy.deepcopy(get_ultimate_layer(model, 'weight')))

In [5]:
classifiers[0].shape

torch.Size([10, 512])

In [6]:
classifiers[0].shape

torch.Size([10, 512])

In [7]:
a = torch.stack(classifiers, dim=0)
a.shape

torch.Size([5, 10, 512])

In [8]:
x = torch.randn([4, 28, 28]).to("cuda")

In [9]:
outputs, representations = model.pred_and_rep(x)

In [10]:
representations.shape

torch.Size([4, 512])

In [52]:
representations.T.shape

torch.Size([512, 4])

In [11]:
c = (a @ representations.T).transpose(1,2)

In [12]:
c.shape

torch.Size([5, 4, 10])

In [13]:
outputs.shape

torch.Size([4, 10])

In [60]:
(c - outputs).shape

torch.Size([5, 4, 10])

In [14]:
func = torch.nn.MSELoss()

In [15]:
func(c, outputs)

/mnt/disk1/anaconda3/envs/longvaipe/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([4, 10])) that is different to the input size (torch.Size([5, 4, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)

In [20]:
func2 = torch.nn.KLDivLoss(reduction='batchmean')

In [28]:
func2(c, outputs.unsqueeze(0))

tensor(-4.4606, device='cuda:0', grad_fn=<DivBackward0>)

In [29]:
d = torch.softmax(c, dim=2)
o = torch.softmax(outputs.unsqueeze(0), dim=2)

In [35]:
func2(o, d)

tensor(-48.0100, device='cuda:0', grad_fn=<DivBackward0>)

In [34]:
o

tensor([[[0.1097, 0.1051, 0.1037, 0.0955, 0.0982, 0.0924, 0.1042, 0.1012,
          0.0898, 0.1003],
         [0.1101, 0.1055, 0.1051, 0.0930, 0.1027, 0.0903, 0.1055, 0.1046,
          0.0847, 0.0985],
         [0.1077, 0.1047, 0.1033, 0.0941, 0.1017, 0.0911, 0.1050, 0.1006,
          0.0892, 0.1028],
         [0.1099, 0.1089, 0.1038, 0.0942, 0.1002, 0.0945, 0.0969, 0.1041,
          0.0887, 0.0988]]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [31]:
torch.sum(d * torch.log(d/o))

tensor(0.0046, device='cuda:0', grad_fn=<SumBackward0>)

In [23]:
x = torch.randn((1,2,5))
x

tensor([[[-1.6104, -3.7056, -1.1230,  0.1467,  0.5184],
         [ 1.3388,  0.3277, -1.5361, -1.3796, -0.0083]]])

In [24]:
torch.softmax(x, dim=2)

tensor([[[0.0590, 0.0073, 0.0960, 0.3419, 0.4958],
         [0.5727, 0.2083, 0.0323, 0.0378, 0.1489]]])

In [4]:
get_module_from_model(model)

[Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
 Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
 Linear(in_features=3136, out_features=512, bias=True),
 Linear(in_features=512, out_features=10, bias=True)]

In [9]:
model1 = Model().to("cuda")
model2 = Model().to("cuda")

In [10]:
a = get_module_from_model(model1)[-1]._parameters['weight']
a

Parameter containing:
tensor([[ 0.0018,  0.0169, -0.0378,  ...,  0.0090,  0.0247, -0.0355],
        [ 0.0194, -0.0076, -0.0395,  ...,  0.0362,  0.0207,  0.0126],
        [ 0.0278,  0.0306,  0.0196,  ...,  0.0350,  0.0006, -0.0363],
        ...,
        [ 0.0311,  0.0039,  0.0292,  ..., -0.0018,  0.0382,  0.0157],
        [ 0.0314,  0.0257,  0.0369,  ..., -0.0113, -0.0016,  0.0050],
        [ 0.0006,  0.0408, -0.0423,  ..., -0.0325, -0.0013,  0.0245]],
       device='cuda:0', requires_grad=True)

In [11]:
b = get_module_from_model(model2)[-1]._parameters['weight']
b

Parameter containing:
tensor([[ 0.0410, -0.0101,  0.0415,  ..., -0.0178, -0.0339, -0.0262],
        [-0.0098, -0.0404,  0.0195,  ...,  0.0100,  0.0086,  0.0101],
        [ 0.0211,  0.0080,  0.0362,  ...,  0.0174, -0.0422,  0.0113],
        ...,
        [-0.0228, -0.0001,  0.0325,  ...,  0.0380,  0.0035,  0.0097],
        [ 0.0050,  0.0420, -0.0077,  ...,  0.0326,  0.0123, -0.0358],
        [-0.0236, -0.0041, -0.0150,  ..., -0.0327, -0.0325,  0.0415]],
       device='cuda:0', requires_grad=True)

In [12]:
with torch.no_grad():
    a.add_(b*2)

In [15]:
with torch.no_grad():
    a.copy_(b)

In [16]:
get_module_from_model(model1)[-1]._parameters['weight']

Parameter containing:
tensor([[ 0.0410, -0.0101,  0.0415,  ..., -0.0178, -0.0339, -0.0262],
        [-0.0098, -0.0404,  0.0195,  ...,  0.0100,  0.0086,  0.0101],
        [ 0.0211,  0.0080,  0.0362,  ...,  0.0174, -0.0422,  0.0113],
        ...,
        [-0.0228, -0.0001,  0.0325,  ...,  0.0380,  0.0035,  0.0097],
        [ 0.0050,  0.0420, -0.0077,  ...,  0.0326,  0.0123, -0.0358],
        [-0.0236, -0.0041, -0.0150,  ..., -0.0327, -0.0325,  0.0415]],
       device='cuda:0', requires_grad=True)

In [1]:
import torch

In [ ]:
a = torch.randn([5,1])

In [5]:
import numpy as np

def pairing_clients(clients, clients_per_group=2):
    """
    clients = [0,1,2,3,4,5,6,....] the list of client's id
    """
    participants = clients.copy()
    pairs = []
    
    while len(participants) > clients_per_group:
        one_pair = list(np.random.choice(participants, clients_per_group, replace=False))
        pairs.append(one_pair)
        participants = list(set(participants) - set(one_pair))
    
    if len(participants):
        pairs.append(participants)
    
    return pairs

In [ ]:
a =[i for i in range(10)]

for i in range(100000):
    x = pairing_clients(a, 2)
    assert len(x) == 5, f"False {x}" 

In [1]:
import numpy as np

In [2]:
a = np.ones([2,5])
a

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [6]:
a = a/np.sum(a, axis=1, keepdims=True)
a

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2]])